In [7]:
'''
@Author: your name
@Date: 2020-04-22 10:09:07
@LastEditTime: 2020-04-22 10:33:16
@LastEditors: Please set LastEditors
@Description: In User Settings Edit
@FilePath: /ctr_practice/model_rewrite.py/cf.py
'''

from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import numpy as np
import pandas as pd

data_root = '/Users/liulingzhi5/dataset/movielens/ml-1m/'
movies_dir = data_root + 'movies.dat'
users_dir = data_root + 'users.dat'
ratings_dir = data_root + 'ratings.dat'

movies = pd.read_csv(movies_dir,sep='::',header=None)
users = pd.read_csv(users_dir,sep='::',header=None)
ratings = pd.read_csv(ratings_dir,sep='::',header=None)
movies.columns=['MovieID', 'Title', 'Genres']
users.columns=['UserID', 'Gender', 'Age', 'Occupation', 'Zip-code']
ratings.columns=['UserID', 'MovieID', 'Rating', 'Timestamp']

# data = ratings.merge(users, on=['UserID'])
# data = data.merge(movies, on=['MovieID'])


In [8]:
# movies数据处理
movies['publish_date'] = movies.Title.str[-5:-1].astype(int)
movies['Title'] = LabelEncoder().fit_transform(movies.Title.str[:-7])

from sklearn.preprocessing import MultiLabelBinarizer
movie_genres = MultiLabelBinarizer().fit_transform(movies.Genres.map(lambda x: x.split('|')))
movie_genres = pd.DataFrame(movie_genres)
movie_genres.columns = ['Genres_%d' % i for i in range(len(movie_genres.columns))]
movies = pd.concat([movies, movie_genres], axis=1)
# users数据处理
users = users.drop(['Zip-code'], axis=1)
users['Gender'] = LabelEncoder().fit_transform(users.Gender)
# ratings数据处理
ratings = ratings.sort_values(['UserID', 'Timestamp'])
ratings['MovieID'] = ratings['MovieID'].astype(str)
watching_seq = ratings.groupby('UserID')['MovieID'].transform(lambda x: ','.join(x))
ratings['MovieID'] = ratings['MovieID'].astype(int)
ratings['watching_seq'] = ratings['UserID'].map(watching_seq)

dt = pd.to_datetime(ratings.Timestamp).dt
ratings['day'] = dt.day
ratings['hour'] = dt.hour
ratings = ratings.drop(['Timestamp'], axis=1)

In [9]:
data = ratings.merge(movies, on='MovieID')
data = data.merge(users, on='UserID')


In [10]:
    data

,UserID,MovieID,Rating,watching_seq,day,hour,Title,Genres,publish_date,Genres_0,Genres_1,Genres_2,Genres_3,Genres_4,Genres_5,Genres_6,Genres_7,Genres_8,Genres_9,Genres_10,Genres_11,Genres_12,Genres_13,Genres_14,Genres_15,Genres_16,Genres_17,Gender,Age,Occupation
0,1,3186,4,"3186,1270,1721,1022,2340,1836,3408,2804,1207,1...",1,0,1380,Drama,1999,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,10
1,1,1270,5,"3186,1270,1721,1022,2340,1836,3408,2804,1207,1...",1,0,264,Comedy|Sci-Fi,1985,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,10
2,1,1721,4,"3186,1270,1721,1022,2340,1836,3408,2804,1207,1...",1,0,3496,Drama|Romance,1997,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,1,10
3,1,1022,5,"3186,1270,1721,1022,2340,1836,3408,2804,1207,1...",1,0,701,Animation|Children's|Musical,1950,0,0,1,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,10
4,1,2340,3,"3186,1270,1721,1022,2340,1836,3408,2804,1207,1...",1,0,2211,Romance,1998,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1000204,4211,3791,2,"957,2936,2366,3690,2922,1136,1210,2948,3507,48...",1,0,1247,Drama,1984,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,45,5
1000205,4211,3840,4,"957,2936,2366,3690,2922,1136,1210,2948,3507,48...",1,0,2783,Horror,1988,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,45,5
1000206,4211,3806,3,"957,2936,2366,3690,2922,1136,1210,2948,3507,48...",1,0,2103,Western,1969,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,45,5
1000207,4211,3766,2,"957,2936,2366,3690,2922,1136,1210,2948,3507,48...",1,0,2271,Action|War,1984,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,45,5


In [11]:
train_data = data[: int(0.8*len(data))]
test_data = data[int(0.8*len(data)):]
# model_input['Genres'] = genres_list


In [30]:
data['Genres']

0                                 Drama
1                         Comedy|Sci-Fi
2                         Drama|Romance
3          Animation|Children's|Musical
4                               Romance
                       ...             
1000204                           Drama
1000205                          Horror
1000206                         Western
1000207                      Action|War
1000208         Adventure|Drama|Romance
Name: Genres, Length: 1000209, dtype: object

In [13]:
# DeepFM
from deepctr.models import DeepFM,AFM,DCN
from deepctr.inputs import SparseFeat, VarLenSparseFeat, DenseFeat, input_from_feature_columns, build_input_features, get_feature_names
from tensorflow.python.keras.preprocessing.sequence import pad_sequences

sparse_features = ['UserID', 'MovieID', 'Title', 'Gender', 'Occupation']
dense_features = ['day', 'hour', 'publish_date', 'Age']
varlen_features = ['Genres']

key2index = {}
def split(x):
    key_ans = x.split('|')
    for key in key_ans:
        if key not in key2index:
            # Notice : input value 0 is a special "padding",so we do not use 0 to encode valid feature for sequence input
            key2index[key] = len(key2index) + 1
    return list(map(lambda x: key2index[x], key_ans))

genres_list = list(map(split, data['Genres'].values))
# genres_list = list(data['Genres'].map(lambda x: x.split('|')).values)
genres_length = np.array(list(map(len, genres_list)))
max_len = max(genres_length)
print(key2index)

# Notice : padding=`post`
genres_list = pad_sequences(genres_list, maxlen=max_len, padding='post', dtype=str, value=0)
feat_vocabulary = {
    'UserID':users['UserID'].max()+1, 
    'MovieID':movies['MovieID'].max()+1, 
    'Title':movies['Title'].max()+1, 
    'Gender':users['Gender'].nunique(), 
    'Occupation':users['Occupation'].nunique()
}

sparse_feature_columns = [SparseFeat(feat, feat_vocabulary[feat], embedding_dim=4, dtype='int64')
                            for feat in sparse_features]
dense_feature_columns = [DenseFeat(feat, 1, ) for feat in dense_features]                            
varlen_feature_columns = [
    VarLenSparseFeat(
        sparsefeat=SparseFeat('Genres', vocabulary_size=len(key2index)+1, embedding_dim=4, dtype="int64"),
        maxlen=max_len,combiner= 'mean')]


# fixlen_feature_columns = sparse_feature_columns + dense_feature_columns
fixlen_feature_columns = sparse_feature_columns + dense_feature_columns + varlen_feature_columns

dnn_feature_columns = fixlen_feature_columns
linear_feature_columns = fixlen_feature_columns 

feature_names = get_feature_names(linear_feature_columns + dnn_feature_columns)


train_input = {name:train_data[name].values for name in feature_names}
train_input['Genres'] = genres_list[: int(0.8*len(data))]
test_input = {name:test_data[name].values for name in feature_names}
test_input['Genres'] = genres_list[int(0.8*len(data)): ]

model = DCN(linear_feature_columns, dnn_feature_columns, task='regression', dnn_use_bn=True)
# model = DeepFM(linear_feature_columns, dnn_feature_columns, task='regression')
# model = AFM(linear_feature_columns, dnn_feature_columns, task='regression')
model.compile("adam", "mse", metrics=['mse'], )
history = model.fit(train_input, train_data['Rating'],
                    batch_size=128, epochs=10, verbose=2, validation_split=0.2, )
y_pred = model.predict(test_input)
mean_squared_error(test_data['Rating'], y_pred)
# 1.190774665808671



{"Children's": 6, 'Drama': 1, 'Comedy': 2, 'Western': 15, 'Adventure': 9, 'Sci-Fi': 3, 'Musical': 7, 'Film-Noir': 17, 'Mystery': 14, 'Fantasy': 10, 'Horror': 16, 'Action': 8, 'Documentary': 18, 'Romance': 4, 'Thriller': 12, 'Animation': 5, 'Crime': 11, 'War': 13}
Train on 640133 samples, validate on 160034 samples
Epoch 1/10
640133/640133 - 33s - loss: 480271296760.2510 - mean_squared_error: 480254263296.0000 - val_loss: 355412152.0757 - val_mean_squared_error: 355411936.0000
Epoch 2/10
640133/640133 - 39s - loss: 64964794.0110 - mean_squared_error: 64964724.0000 - val_loss: 762755820.5329 - val_mean_squared_error: 762755392.0000
Epoch 3/10
640133/640133 - 49s - loss: 178138.8914 - mean_squared_error: 178139.0312 - val_loss: 61974211.8859 - val_mean_squared_error: 61974232.0000
Epoch 4/10
640133/640133 - 59s - loss: 188432.5763 - mean_squared_error: 188432.6562 - val_loss: 37498398.9426 - val_mean_squared_error: 37498376.0000
Epoch 5/10
640133/640133 - 54s - loss: 160147.6032 - mean_sq

389624.465888532

In [17]:
test_data['Rating']

800167     3
800168     3
800169     3
800170     4
800171     5
          ..
1000204    2
1000205    4
1000206    3
1000207    2
1000208    2
Name: Rating, Length: 200042, dtype: int64

In [36]:
train_input['Genres']

array([['1', '0', '0', '0', '0', '0'],
       ['2', '3', '0', '0', '0', '0'],
       ['1', '4', '0', '0', '0', '0'],
       ...,
       ['5', '6', '2', '7', '0', '0'],
       ['5', '6', '2', '0', '0', '0'],
       ['5', '6', '2', '0', '0', '0']], dtype='<U1')

In [16]:
# GBDT
from sklearn.ensemble import GradientBoostingRegressor

# gbm = lgb.LGBMRegressor()
gbm = GradientBoostingRegressor()
x = train_data[feature_names]
y = train_data['Rating']
gbm.fit(x, y)

y_pred = gbm.predict(test_data[feature_names])
mean_squared_error(test_data['Rating'], y_pred)
# 1.139487247280116


1.139487247280116

In [13]:
from deepctr.models import DIN
from deepctr.inputs import get_feature_names, build_input_features, SparseFeat, DenseFeat, VarLenSparseFeat
from tensorflow.python.keras.preprocessing.sequence import pad_sequences

key2index = {}




def get_xy_fd():

    sparse_features = ['UserID', 'MovieID', 'Title', 'Gender', 'Occupation']
    dense_features = ['day', 'hour', 'publish_date', 'Age']
    varlen_features = ['Genres', 'watching_seq']


    sparse_feature_columns = [SparseFeat(name=feat,vocabulary_size=data[feat].nunique(), embedding_dim=4, dtype='int64') for feat in sparse_features]
    dense_feature_columns = [DenseFeat(feat, 1 ,dtype='float64') for feat in dense_features]


    def split(x):
        key_ans = x.split('|')
        for key in key_ans:
            if key not in key2index:
                # Notice : input value 0 is a special "padding",so we do not use 0 to encode valid feature for sequence input
                key2index[key] = len(key2index) + 1
        return list(map(lambda x: key2index[x], key_ans))


    genres_list = list(map(split, data['Genres'].values))
    genres_length = np.array(list(map(len, genres_list)))
    max_len = max(genres_length)
    # Notice : padding=`post`
    genres_list = pad_sequences(genres_list, maxlen=max_len, padding='post', )
    varlen_feature_columns = [
        VarLenSparseFeat(
            sparsefeat=SparseFeat('Genres', vocabulary_size=len(key2index) + 1, embedding_dim=4, dtype="int64"), 
            maxlen=max_len,
            combiner= 'mean'
        )
    ]
    print(key2index)

    def split(x):
        key_ans = x.split(',')
        for key in key_ans:
            if key not in key2index:
                # Notice : input value 0 is a special "padding",so we do not use 0 to encode valid feature for sequence input
                key2index[key] = len(key2index) + 1
        return list(map(lambda x: key2index[x], key_ans))
    
    watching_seq_list = list(map(split, data['watching_seq'].values))
    watching_seq_length = np.array(list(map(len, watching_seq_list)))
    max_len = max(watching_seq_length)
    # Notice : padding=`post`
    watching_seq_list = pad_sequences(watching_seq_list, maxlen=max_len, padding='post', )
    varlen_feature_columns = [
        VarLenSparseFeat(
            sparsefeat=SparseFeat('hist_MovieID', vocabulary_size=len(key2index) + 1, embedding_dim=4, dtype="int64"), 
            maxlen=max_len,
        )
    ]
    print(key2index)


    feature_columns = sparse_feature_columns + dense_feature_columns + varlen_feature_columns
    

    behavior_feature_list = ['MovieID']

    feature_dict = {
        'UserID': data['UserID'], 'MovieID': data['MovieID'], 'Gender': data['Gender'], 'Title':data['Title'], 'Occupation':data['Occupation'],
        'day': data['day'], 'hour': data['hour'], 'publish_date': data['publish_date'], 'Age': data['Age'],
        'Genres': genres_list,
        'hist_MovieID': watching_seq_list
    }
    
    x = {name:feature_dict[name] for name in get_feature_names(feature_columns)}
    y = data['Rating']
    return x, y, feature_columns, behavior_feature_list

x, y, feature_columns, behavior_feature_list = get_xy_fd()
model = DIN(feature_columns, behavior_feature_list, task='regression')
model.compile('adam', 'mse',
                metrics=['mse'])
history = model.fit(x, y, verbose=1, epochs=10, validation_split=0.5)



 '232': 1066, '2436': 670, '3835': 1157, '1185': 907, '1003': 1395, '3534': 176, '3543': 1337, '595': 43, '2935': 1502, '2732': 1602, '1258': 1099, '62': 595, '1887': 793, '1216': 841, '3429': 1507, '2496': 596, '2638': 1100, '907': 1533, '555': 430, '1603': 1656, '2239': 1594, '3347': 769, '1704': 269, '528': 1439, '2047': 678, '3198': 573, '1721': 3, '30': 920, '2236': 109, '2340': 5, '1077': 947, '2419': 1206, '1090': 71, '3612': 1728, '3728': 274, '2546': 1745, '2083': 1050, '2265': 1160, '2953': 1055, '3178': 506, '605': 1424, '1179': 931, '1060': 532, '945': 1614, '3502': 1342, '608': 14, '3584': 1387, '1517': 323, '556': 1232, '442': 162, '1135': 676, '2986': 1700, '434': 173, '3114': 34, '2724': 861, '330': 1116, '2529': 589, '3016': 218, '1756': 650, '837': 1580, '1257': 606, '3363': 620, '1653': 343, '231': 811, '3406': 1689, '1588': 1340, '2100': 383, '58': 447, '2321': 36, '1303': 1732, '2926': 1532, '1219': 1104, '1672': 876, '1614': 909, '913': 232, '2338': 1085, '1393': 

ValueError: A `Concatenate` layer should be called on a list of at least 2 inputs